In [38]:
bucket = 'ml-lab-pyspark'
prefix = 'sagemaker/tuning/linear_learner'
 
# Define IAM role
import boto3
from sagemaker import get_execution_role

In [39]:
boto3_session=boto3.Session(profile_name='ml-lab')
sagemaker_client=boto3_session.client('sagemaker')

try:
    role = get_execution_role()
except:
    role="arn:aws:iam::814948925568:role/service-role/AmazonSageMaker-ExecutionRole-20200310T154729"

In [40]:
from sagemaker.inputs import s3_input
training_data='s3://{}/{}'.format(bucket, 'training_data.io')
test_data='s3://{}/{}'.format(bucket, 'test_data.io')
validation_data='s3://{}/{}'.format(bucket, 'validation_data.io')


In [50]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(boto3.Session().region_name, 'linear-learner')

In [54]:
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [
        { 
            "Name": "use_bias",
            "Values": [ "true","false" ]
        }
      ],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0.0000001",
          "ScalingType":"Logarithmic",
          "Name": "l1"
        },
        {
          "MaxValue": "1",
          "MinValue": "0.00001",
          "ScalingType":"Logarithmic",
          "Name": "learning_rate"
        },
        {
          "MaxValue": "10000",
          "MinValue": "0.00001",
          "ScalingType":"Logarithmic",
          "Name": "positive_example_weight_mult"
        },
        {
          "MaxValue": "1",
          "MinValue": "0.0000001",
          "ScalingType":"Logarithmic",
          "Name": "wd"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "1000",
          "MinValue": "100",
          "ScalingType":"Logarithmic",
          "Name": "mini_batch_size"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 10,
      "MaxParallelTrainingJobs": 2
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "test:binary_f_beta",
      "Type": "Maximize"
    }
  }

In [55]:
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": training_image,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": training_data
          }
        }
      },
      {
        "ChannelName": "test",
        "CompressionType": "None",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": test_data
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": validation_data
          }
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,prefix)
    },
    "ResourceConfig": {
      "InstanceCount": 1,
      "InstanceType": "ml.c4.xlarge",
      "VolumeSizeInGB": 10
    },
    "RoleArn": role,
    "StaticHyperParameters": {
      "feature_dim": "605",
      "epochs": "15",
      "normalize_data":"true",
      "normalize_label":"false",
      "predictor_type":"binary_classifier",
      "binary_classifier_model_selection_criteria":"precision_at_target_recall",
      "target_recall":"0.25"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}

In [60]:
tuning_job_name = "tuning-relative-price-f-beta"
sagemaker_client.create_hyper_parameter_tuning_job( HyperParameterTuningJobName = tuning_job_name,
                                                    HyperParameterTuningJobConfig = tuning_job_config,
                                                    TrainingJobDefinition = training_job_definition)

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:eu-west-2:814948925568:hyper-parameter-tuning-job/tuning-relative-price-f-beta',
 'ResponseMetadata': {'RequestId': '6dece92f-660a-404a-b85d-e7928ecba913',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6dece92f-660a-404a-b85d-e7928ecba913',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '129',
   'date': 'Mon, 16 Mar 2020 19:37:16 GMT'},
  'RetryAttempts': 0}}